In [1]:
from keras.models import load_model
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import keras
import os
import torch
import numpy as np
import cv2
import pafy
from time import time
from keras.models import load_model
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import keras
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torch.nn.functional as F
import torchvision
#Tools lib
import numpy as np
import cv2
import random
import time
import os
import argparse
from models_custom import *
from skimage.metrics import structural_similarity as ssim
from statistics import mean
from vif_utils import vif, fsim
from CVcheck import ObjectDetection
from Denoise import StreamDenoise
import argparse
from metrics import calc_psnr, calc_ssim

from config import global_config

CFG = global_config.cfg

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Available device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
print('Using CPU..')

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-f')
    parser.add_argument("--type", type=str, default='images') #it can be video or images
    parser.add_argument("--mode", type=str, default='evaluate') #it can be evaluate or demo
    parser.add_argument("--object_detection_test", type=bool, default=True)
    parser.add_argument("--input", type=str, default='./input')
    parser.add_argument("--arch", type=str, default='CAE') #it can be MMAE, GAN
    parser.add_argument("--rain", type=str, default='rainstreaks') #it can be rainstreaks or raindrops
    parser.add_argument("--output_dir", type=str, default='./output')
    parser.add_argument("--weights", type=str, default='./weights')
    parser.add_argument("--enhancer", type=str, default='GAN')# it can be GAN or Autoencoder

    args = parser.parse_args()
    return args


Available device: cuda
Tesla M10
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
Using CPU..


/home/hamroua/.local/lib/python3.6/site-packages/phasepack/tools.py:14: UserWarning: 
Module 'pyfftw' (FFTW Python bindings) could not be imported. To install it, try
running 'pip install pyfftw' from the terminal. Falling back on the slower
'fftpack' module for 2D Fourier transforms.
  'fftpack' module for 2D Fourier transforms.""")


In [2]:
args = get_args()

print('Testing Denoising Model: ', args.arch )
print('Images are: ', args.rain )
if args.arch=='CAE':
    print('Enhancer is: ', args.enhancer )
    if args.rain=='rainstreaks':
        denoisingModel = load_model(args.weights+"/rainstreaks/DerainAdvanced.hdf5",compile=False)
    else:
        denoisingModel = load_model(args.weights+"/raindrops/post-trained.hdf5",compile=False)
    if args.enhancer=='GAN':
        enhancerModel = load_model(args.weights+"/GAN_enhancer.h5",compile=False)
    else:
        enhancerModel = load_model(args.weights+"/AE_Enhancer.hdf5",compile=False)
elif args.arch=='GAN':
    denoisingModel = Generator().cpu()
    denoisingModel.load_state_dict(torch.load(args.weights+"/raindrops/GAN_generator.pkl"))
    enhancerModel=denoisingModel

Testing Denoising Model:  CAE
Images are:  rainstreaks
Enhancer is:  GAN


In [3]:


if args.type=='images':
    print('Loading test images...')
    denoise = StreamDenoise(args.input+'/images',args.output_dir)
    denoise('images',denoisingModel,enhancerModel,args.arch)
    print('Image denoising completed!')
    print('Image denoising files are saved in ',args.output_dir+"/images/")
    if args.object_detection_test==True:
        print('Running Computer vision algorithms for detection for images before denoising...')
        check = ObjectDetection(args.input+'/images',args.output_dir+"/precvimages")
        check('images')
        print('Computer vision algorithms for detection completed!')
        print('Resultant images are saved in: ',args.output_dir+"/precvimages")
           
        print('Running Computer vision algorithms for detection for images after denoising...')
        check = ObjectDetection(args.output_dir+'/images',args.output_dir+"/imagesaftercv")
        check('images')
        print('Computer vision algorithms for detection completed!')
        print('Resultant images are saved in: ',args.output_dir+"/imagesaftercv")


elif args.type=='video': 
    print('Treating video...')
    fourcc = cv2.VideoWriter_fourcc(*'mp4a')
    print(fourcc)
    denoise = StreamDenoise(args.input+'/video/input.mp4',args.output_dir)
    denoise('video',denoisingModel,enhancerModel,args.arch)
    print('Video denoising completed! Files are saved in ',args.output_dir+"/videos/result.mp4")
    if args.object_detection_test==True:
        print('Running Computer vision algorithms for detection for videos after  denoising...')
        check = ObjectDetection(args.output_dir+'/video/input.mp4',args.output_dir)
        check('video')
        print('Computer vision algorithms for detection completed!')
        
if args.mode=='evaluate':
    avgpsnr=0
    avgssim=0
    mse = []
    vifr = []
    vifb = []
    vifg = []
    FIM=[]
    
    for i in glob.glob(args.input+"/images/*.png"):
        print('Evaluating images: ',i)
        print(os.path.basename(i))
        label_image_vis_gray  = cv2.resize(
            cv2.imread(i, cv2.IMREAD_COLOR), (CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT), interpolation=cv2.INTER_LINEAR
        )
        output_image_gray = cv2.resize(
            cv2.imread(args.output_dir+"/derained_images/no"+os.path.basename(i), cv2.IMREAD_COLOR), (CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT), interpolation=cv2.INTER_LINEAR
        )
        
        psnr = calc_psnr(label_image_vis_gray, output_image_gray)
        ssim = calc_ssim(label_image_vis_gray, output_image_gray)
        avgpsnr+=psnr
        avgssim+=ssim
        print('SSIM: {:.5f}'.format(ssim))
        print('PSNR: {:.5f}'.format(psnr))
        mse.append(((label_image_vis_gray - output_image_gray) ** 2).mean(axis=None))
        vifr.append(vif(label_image_vis_gray[:, :, 0].reshape(CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT), output_image_gray[:, :, 0].reshape(CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT)))
        vifg.append(vif(label_image_vis_gray[:, :, 1].reshape(CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT), output_image_gray[:, :, 1].reshape(CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT)))
        vifb.append(vif(label_image_vis_gray[:, :, 2].reshape(CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT), output_image_gray[:, :, 2].reshape(CFG.TEST.IMG_WIDTH, CFG.TEST.IMG_HEIGHT)))
        FIM.append(fsim(label_image_vis_gray, output_image_gray))
    print("Average MSE", mean(mse))
    print("Average VIF", mean(vifr + mean(vifg) + mean(vifb)) / 3)
    print("Average FSIM", mean(FIM))   
    print('Average SSIM {:.5f}'.format(avgssim/len(os.listdir(args.input+"/images"))))
    print('Average PSNR {:.5f}'.format(avgpsnr/len(os.listdir(args.input+"/images"))))

    
            



Loading test images...
Loading images:  ./input/images/rain-001.png
Loading images:  ./input/images/rain-002.png
Loading images:  ./input/images/rain-003.png
Loading images:  ./input/images/rain-004.png
Loading images:  ./input/images/rain-005.png
Loading images:  ./input/images/rain-006.png
Loading images:  ./input/images/rain-007.png
Loading images:  ./input/images/rain-008.png
Loading images:  ./input/images/rain-009.png
Loading images:  ./input/images/rain-010.png
Loading images:  ./input/images/rain-011.png
Loading images:  ./input/images/rain-012.png
Loading images:  ./input/images/rain-013.png
Loading images:  ./input/images/rain-014.png
Loading images:  ./input/images/rain-015.png
Loading images:  ./input/images/rain-016.png
Loading images:  ./input/images/rain-017.png
Loading images:  ./input/images/rain-018.png
Loading images:  ./input/images/rain-019.png
Loading images:  ./input/images/rain-020.png
Loading images:  ./input/images/rain-021.png
Loading images:  ./input/images/

/home/hamroua/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Processing image: rain-002.png
Processing image: rain-003.png
Processing image: rain-004.png
Processing image: rain-005.png
Processing image: rain-006.png
Processing image: rain-007.png
Processing image: rain-008.png
Processing image: rain-009.png
Processing image: rain-010.png
Processing image: rain-011.png
Processing image: rain-012.png
Processing image: rain-013.png
Processing image: rain-014.png
Processing image: rain-015.png
Processing image: rain-016.png
Processing image: rain-017.png
Processing image: rain-018.png
Processing image: rain-019.png
Processing image: rain-020.png
Processing image: rain-021.png
Processing image: rain-022.png
Processing image: rain-023.png
Processing image: rain-024.png
Processing image: rain-025.png
Processing image: rain-026.png
Processing image: rain-027.png
Processing image: rain-028.png
Processing image: rain-029.png
Processing image: rain-030.png
Processing image: rain-031.png
Processing image: rain-032.png
Processing image: rain-033.png
Processi

Using cache found in /home/hamroua/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-26 torch 1.7.1+cu110 CUDA:0 (Tesla M10, 8129MiB)



Python 3.7.0 required by YOLOv5, but Python 3.6.9 is currently installed


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
/home/hamroua/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
Using cache found in /home/hamroua/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-26 torch 1.7.1+cu110 CUDA:0 (Tesla M10, 8129MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Computer vision algorithms for detection completed!
Resultant images are saved in:  ./output/precvimages
Running Computer vision algorithms for detection for images after denoising...
Python 3.7.0 required by YOLOv5, but Python 3.6.9 is currently installed
Computer vision algorithms for detection completed!
Resultant images are saved in:  ./output/imagesaftercv
Evaluating images:  ./input/images/rain-001.png
rain-001.png
SSIM: 0.83546
PSNR: 26.44704


/home/hamroua/.local/lib/python3.6/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


Evaluating images:  ./input/images/rain-002.png
rain-002.png
SSIM: 0.81988
PSNR: 28.09006
Evaluating images:  ./input/images/rain-003.png
rain-003.png
SSIM: 0.92594
PSNR: 33.14771
Evaluating images:  ./input/images/rain-004.png
rain-004.png
SSIM: 0.93605
PSNR: 33.53925
Evaluating images:  ./input/images/rain-005.png
rain-005.png
SSIM: 0.63551
PSNR: 20.65484
Evaluating images:  ./input/images/rain-006.png
rain-006.png
SSIM: 0.83104
PSNR: 26.26064
Evaluating images:  ./input/images/rain-007.png
rain-007.png
SSIM: 0.71611
PSNR: 20.80372
Evaluating images:  ./input/images/rain-008.png
rain-008.png
SSIM: 0.68406
PSNR: 20.76015
Evaluating images:  ./input/images/rain-009.png
rain-009.png
SSIM: 0.94496
PSNR: 30.75485
Evaluating images:  ./input/images/rain-010.png
rain-010.png
SSIM: 0.83069
PSNR: 25.28644
Evaluating images:  ./input/images/rain-011.png
rain-011.png
SSIM: 0.73396
PSNR: 22.99272
Evaluating images:  ./input/images/rain-012.png
rain-012.png
SSIM: 0.79217
PSNR: 24.63475
Evaluating

Evaluating images:  ./input/images/rain-094.png
rain-094.png
SSIM: 0.79289
PSNR: 25.96232
Evaluating images:  ./input/images/rain-095.png
rain-095.png
SSIM: 0.66266
PSNR: 21.68980
Evaluating images:  ./input/images/rain-096.png
rain-096.png
SSIM: 0.76115
PSNR: 26.48627
Evaluating images:  ./input/images/rain-097.png
rain-097.png
SSIM: 0.88674
PSNR: 25.93664
Evaluating images:  ./input/images/rain-098.png
rain-098.png
SSIM: 0.91709
PSNR: 33.68474
Evaluating images:  ./input/images/rain-099.png
rain-099.png
SSIM: 0.91625
PSNR: 25.92097
Evaluating images:  ./input/images/rain-100.png
rain-100.png
SSIM: 0.88576
PSNR: 27.03327
Average MSE 24.353301195987655
Average VIF 0.36465881883195905
Average FSIM 0.7840632451989422
Average SSIM 0.81475
Average PSNR 26.09138
